Задача:

Для случайных СЛАУ вида $Ax = b$, размерности $n$, найти обратную матрицу, добавив к ней единичную той же размерности и выполнив над расширенной матрицей преобразованиее Гаусса-Жордана. $(A|E) \rightarrow (E|A^{-1})$

Протетстировать обратную мтарицу, которая сформируется на месте добавленной единичной матрицы, затем найти решение через эту обратную матрицу. ($x = A^{-1}b$)

Далее протестировать полученное решение, найдя для него невзяку $dx = b - Ax$.

Теория всё та же, что и с методом Гаусса, что очень тривиальна в доказательстве. Потому здесь я даже для себя расписывать ничего не буду. Реализация метода абсолютно такая же, если не проще. Асимптотика решения остаётся всё той же $O(N^3)$.

Зато я научусь находить обратную матрицу самостоятельно. Теперь я точно хочу реализовать хотя бы метод гаусса на плюсах.

In [1]:
import numpy as np
from typing import Tuple
from sympy import Matrix, init_printing
init_printing()

def isEqual(x, y) -> bool:
  return np.isclose(x, y, rtol=1e-05, atol=1e-08, equal_nan=False)

def GenerateMatrixOnes(N: int) -> Tuple[np.ndarray, np.ndarray]:
  Matr = np.full((N, N), 0.5) - np.random.rand(N, N)
  A = np.dot(Matr.T, Matr)
  b = np.zeros((N, 1))
  for i in range(N):
    b[i] += np.sum(A[i, :])

  return (A, b)

def GaussianElimination(A: np.ndarray) -> Tuple[np.ndarray, int]:
  N: int = A.shape[0]
  C = A.copy()
  rank: int = 1

  for i in range(0, N - 1):
    # если на гл. диагонали возник ноль, то поднимаем наверх первую строку
    # с ненулевым значением на i-том столбце
    if np.abs(C[i, i]) == 0:
      for k in range(i + 1, N):
        if np.abs(C[k, i]) > np.abs(C[i, i]):
          C[[i, k]] = C[[k, i]]
          break

    # если же весь нижний столбец нулевой, то пропускаем шаг
    if (isEqual(C[i, i], 0)):
      continue

    rank += 1
    # сам метод Гаусса
    for j in range(i + 1, N):
      m = C[j, i] / C[i, i]
      C[j, : ] = C[j, : ] - m * C[i, : ]

  return (C, rank)

def StepBack(C: np.ndarray) -> None:
  N = C.shape[0]

  for i in range(N - 1, -1, -1):
    C[i] /= C[i, i]
    for j in range(i - 1, -1, -1):
      C[j] -= C[j, i] * C[i]

Нахождение обратной матрицы и вывод требуемых в лабораторной работе параметров.

In [3]:
N: int = 4

A, b = GenerateMatrixOnes(N)
E = np.eye(N)
print("Сгенерированная матрица")
display(Matrix(A))

C = A.copy()
expanded = np.concatenate([C, E], axis=1)
print("Расширенная матрица")
display(Matrix(expanded))

GE_exp, rank = GaussianElimination(expanded)

print(f"Ранг матрицы = {rank}")
if rank == N:
  StepBack(GE_exp)

invA = np.split(GE_exp, 2, axis=1)[1]

x = np.matmul(invA, b)
dx = b - np.matmul(A, x)


print("Расширенная матрица после метода Гаусса")
display(Matrix(GE_exp))
print("Обратная матрица")
print(invA)
display(Matrix(invA))
print("Обратная матрица функцией invert")
display(Matrix(np.linalg.inv(A)))
print("Проверка А * А^-1")
display(Matrix(np.matmul(A, invA)))
print("Подсчет невязки")
display(Matrix(dx))

Сгенерированная матрица


⎡ 0.46857096610331   -0.106786707633028   -0.189007711236203    0.193514846011
⎢                                                                             
⎢-0.106786707633028   0.111576838619592   -0.0289851790022163  0.0072777658620
⎢                                                                             
⎢-0.189007711236203  -0.0289851790022163   0.353696376795842   -0.187011987178
⎢                                                                             
⎣0.193514846011087   0.00727776586208754  -0.187011987178556    0.304168484082

087 ⎤
    ⎥
8754⎥
    ⎥
556 ⎥
    ⎥
137 ⎦

Расширенная матрица


⎡ 0.46857096610331   -0.106786707633028   -0.189007711236203    0.193514846011
⎢                                                                             
⎢-0.106786707633028   0.111576838619592   -0.0289851790022163  0.0072777658620
⎢                                                                             
⎢-0.189007711236203  -0.0289851790022163   0.353696376795842   -0.187011987178
⎢                                                                             
⎣0.193514846011087   0.00727776586208754  -0.187011987178556    0.304168484082

087   1.0   0    0    0 ⎤
                        ⎥
8754   0   1.0   0    0 ⎥
                        ⎥
556    0    0   1.0   0 ⎥
                        ⎥
137    0    0    0   1.0⎦

Ранг матрицы = 4
Расширенная матрица после метода Гаусса


...

⎡ 5.1094656619022   5.56563232110262   2.07035912924584  -2.11093814230431⎤
⎢                                                                         ⎥
⎢5.56563232110262   15.2464391772032   3.19817384480496  -1.93937137128248⎥
⎢                                                                         ⎥
⎢2.07035912924584   3.19817384480496   5.12481543951571  1.75718773878773 ⎥
⎢                                                                         ⎥
⎣-2.11093814230431  -1.93937137128248  1.75718773878773  5.75742531797072 ⎦

Проверка А * А^-1


⎡         1.0           -4.86798644966099e-16  8.87409125808613e-17   -2.74366
⎢                                                                             
⎢-5.45160511466362e-17           1.0           -5.93746005039595e-17  1.592488
⎢                                                                             
⎢-7.37719736161562e-17  1.59649722099049e-16            1.0           -4.33690
⎢                                                                             
⎣-6.5819085082308e-17   -2.65692153045423e-16  -9.46396068378337e-17          

163806356e-17⎤
             ⎥
36316219e-17 ⎥
             ⎥
415693588e-17⎥
             ⎥
 1.0         ⎦

Подсчет невязки


⎡-5.55111512312578e-17⎤
⎢                     ⎥
⎢-1.38777878078145e-17⎥
⎢                     ⎥
⎢8.32667268468867e-17 ⎥
⎢                     ⎥
⎣-5.55111512312578e-17⎦